In [27]:
import datetime as dt
import json
import time
import random
from datetime import datetime
import traceback
from tqdm import tqdm

from bitshares import BitShares
from bitshares.account import Account
from bitshares.wallet import Wallet
from bitsharesbase.account import BrainKey
from bitshares.blockchain import Blockchain
from bitsharesbase.chains import known_chains

In [28]:
web_socket = "ws://testnet-seed-7.gravityprotocol.org:4624"
chain_id = "ab5071857c28ddbc872d0ca508725fa3006ea7bdfda10f707433021f570fc27e" 
known_chains = { "ZGV": {"chain_id": "ab5071857c28ddbc872d0ca508725fa3006ea7bdfda10f707433021f570fc27e",
                         "core_symbol": "ZGV",
                         "prefix": "ZGV"}}

testnet = BitShares(web_socket)
bc = Blockchain(testnet)
users = list(bc.get_all_accounts())

/Users/mac/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: generator 'Blockchain.get_all_accounts' raised StopIteration
  if __name__ == '__main__':


In [20]:
import csv

In [23]:
pre_emission_data = []
with open('testnet_first_emission/results/emission_2018-06-01_12-39-48.csv', newline='') as file:
    r = csv.reader(file, delimiter=';')
    for row in file:
        row = row.strip().split(';')
        pre_emission_data.append(row)

In [32]:
len(pre_emission_data)

1262

In [30]:
#Словари между id и именем аккаунта
user_ids = {}
user_names = {}
for u in users:
    try:
        acc = Account(u, bitshares_instance=testnet)
        user_ids[u] = acc['id']
        user_names[acc['id']] = u
    except:
        print(u)

In [31]:
#Поиск родительских нод по ключу registrar 
parents = []
kids = []
for user in tqdm(users):
    real_acc = Account(user, bitshares_instance=testnet)
    h = real_acc.history(first=None,last=0,limit=1000000,only_ops=[],exclude_ops=[])
    hista = list(h)
    #print(hista[-1]['op'][1])
    try:
        #print(hista[-1]['op'][1]['registrar'])
        #if hista[-1]['op'][1]['registrar'] not in parents:
        parents.append(hista[-1]['op'][1]['registrar'])
        kids.append(user)
        
    except:
        pass


 99%|█████████▉| 2377/2394 [11:39<00:05,  3.40it/s]/Users/mac/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: generator 'Account.history' raised StopIteration
  import sys
100%|██████████| 2394/2394 [11:41<00:00,  3.41it/s]


In [33]:
pars = [user_names[p] for p in set(parents)]

In [34]:
# Словарь родитель и список его нод
parent_accounts = {}
for p in parents:
    name = user_names[p]
    parent_accounts[name] = []

In [35]:
for user in tqdm(users):
    real_acc = Account(user, bitshares_instance=testnet)
    h = real_acc.history(first=None,last=0,limit=1000000,only_ops=[],exclude_ops=[])
    hista = list(h)
    try:
        p = user_names[hista[-1]['op'][1]['registrar']]
        parent_accounts[p].append(user)
    except:
        pass

 99%|█████████▉| 2377/2394 [11:40<00:05,  3.40it/s]/Users/mac/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: generator 'Account.history' raised StopIteration
  after removing the cwd from sys.path.
100%|██████████| 2394/2394 [11:42<00:00,  3.41it/s]


In [36]:
parent_accounts

{'g2390w9350c1220p0270': ['g0039r8540v5126z0390',
  'g0040r2052v2763z7534',
  'g0068r2258v0455z8811',
  'g0068r2570v5802z0738',
  'g0074r0953v2455z5851',
  'g0092r1465v1172z2122',
  'g0102r9301v8327z0629',
  'g0106r2247v0024z4048',
  'g0109r8084v4921z9300',
  'g0144r3422v6964z0375',
  'g0216r3174v8839z1714',
  'g0221r3893v5766z6163',
  'g0253r4379v3248z8223',
  'g0271r7822v1468z5089',
  'g0282r1539v9120z4597',
  'g0283r0816v3234z5189',
  'g0303r3417v5008z0185',
  'g0318r2425v1800z0012',
  'g0323r2536v5603z5610',
  'g0323r5225v2310z6816',
  'g0327r0572v2215z5392',
  'g0329r1333v8306z9520',
  'g0355r1427v4742z4392',
  'g0387r9522v6778z5518',
  'g0404r2366v5487z8991',
  'g0404r5459v7530z0826',
  'g0406r7732v7754z3902',
  'g0416r5809v3279z3184',
  'g0429r2655v6956z2359',
  'g0432r2677v6546z2562',
  'g0434r9029v2895z5475',
  'g0437r2572v1415z4366',
  'g0444r7138v1036z8904',
  'g0461r8330v8638z5740',
  'g0462r0282v8216z6158',
  'g0475r0486v2117z8177',
  'g0477r0533v0992z3291',
  'g0497r3554v

In [37]:
#Родительская ноды и кол-во её акков
for p in set(parents):
    print(p, user_names[p], parents.count(p))

1.2.242 g9480a2230f7280f3790 1001
1.2.17 nathan 408
1.2.212 g8090b6110c6310f7000 99
1.2.259 g2390w9350c1220p0270 812
1.2.199 g9320b3400s1180y4450 12


In [39]:
groups = {}
groups['nathan'] = 'nathan'
groups['faucet'] = 'faucet'
groups['trees'] = [[key]+parent_accounts[key] for key in parent_accounts.keys()]
groups['secluded'] = [u for u in users if u not in pars and u not in kids and u != 'nathan' and u != 'faucet']

In [51]:
pre_emission_data[1]

['faucet',
 '54087161799593',
 '0',
 '0.00361538',
 '0.000362',
 '653667381638.169800']

In [81]:
emission_balances = {}
total_sum = 0
for u in pre_emission_data:
    total_sum += float(u[1])/100000
    emission_balances[u[0]] = float(u[1])/100000

In [87]:
#Полный суммарный баланс перед эмиссией
total_sum

999387634.6106406

In [90]:
total_activity = 0
activities = {}
for u in pre_emission_data:
    total_activity += float(u[3])
    activities [u[0]] = float(u[3])

In [91]:
total_activity

0.9999995999999993

In [92]:
total_gravity = 0
gravity = {}
for u in pre_emission_data:
    total_gravity += float(u[4])
    gravity[u[0]] = float(u[4])

In [93]:
total_gravity

0.09998599999999978

In [95]:
total_emission = 0
actual_emission = {}
for u in pre_emission_data:
    total_emission += float(u[5])/100000
    actual_emission[u[0]] = float(u[5])/100000

In [96]:
total_emission

13410962.58352171

In [63]:
#Те кто получил эмиссию
e_users = [u[0] for u in pre_emission_data]

In [67]:
#Балансы по группам
balance_groups = {}
balance_groups['faucet'] = emission_balances['faucet']
balance_groups['nathan'] = emission_balances['nathan']
balance_groups['secluded'] = 0
for key in emission_balances.keys():
    if key != 'nathan' and key != 'faucet' and key not in kids and key not in pars:
        balance_groups['secluded'] += emission_balances[key]

In [80]:
balance_groups['trees'] = {}
for key in emission_balances.keys():
    if key in pars:
        balance_groups['trees'][key] = emission_balances[key]
for key in balance_groups['trees'].keys():
    for k in parent_accounts[key]:
        try:
            balance_groups['trees'][key] += emission_balances[k]
        except:
            pass

In [82]:
balance_groups['trees']

{'g2390w9350c1220p0270': 672683.26749,
 'g8090b6110c6310f7000': 1599077.71038,
 'g9320b3400s1180y4450': 101549.77652000007,
 'g9480a2230f7280f3790': 56748852.87191,
 'nathan': 400116867.59451026}

In [68]:
activity_groups = {}
activity_groups['faucet'] = activities['faucet']
activity_groups['nathan'] = activities['nathan']
activity_groups['secluded'] = 0
for key in activities.keys():
    if key != 'nathan' and key != 'faucet' and key not in kids and key not in pars:
        activity_groups['secluded'] += activities[key]

In [88]:
activity_groups['trees'] = {}
for key in activities.keys():
    if key in pars:
        activity_groups['trees'][key] = activities[key]
for key in activity_groups['trees'].keys():
    for k in parent_accounts[key]:
        try:
            activity_groups['trees'][key] += activities[k]
        except:
            pass

In [97]:
activity_groups

{'faucet': 0.00361538,
 'nathan': 0.00270099,
 'secluded': 0.00866641,
 'trees': {'g2390w9350c1220p0270': 0.00128664,
  'g8090b6110c6310f7000': 0.00135185,
  'g9320b3400s1180y4450': 0.00135185,
  'g9480a2230f7280f3790': 0.00982202,
  'nathan': 0.9766091499999993}}

In [70]:
gravity_groups = {}
gravity_groups['faucet'] = gravity['faucet']
gravity_groups['nathan'] = gravity['nathan']
gravity_groups['secluded'] = 0
for key in gravity.keys():
    if key != 'nathan' and key != 'faucet' and key not in kids and key not in pars:
        gravity_groups['secluded'] += gravity[key]

In [84]:
gravity_groups['trees'] = {}
for key in gravity.keys():
    if key in pars:
        gravity_groups['trees'][key] = gravity[key]
for key in gravity_groups['trees'].keys():
    for k in parent_accounts[key]:
        try:
            gravity_groups['trees'][key] += gravity[k]
        except:
            pass

In [86]:
gravity_groups['trees']

{'g2390w9350c1220p0270': 0.000129,
 'g8090b6110c6310f7000': 0.000135,
 'g9320b3400s1180y4450': 0.000135,
 'g9480a2230f7280f3790': 0.000982,
 'nathan': 0.09764799999999976}